In [2]:
import openai
import json
import copy
from set_key import set_openai_api_key_from_txt

In [2]:
set_openai_api_key_from_txt('./key/[].txt')

In [3]:
AGENT_NAME = {
    1:"cooking", 2:"cleaning", 3:"massaging"
}
PROMPT_STARTER = {
    1: "I am a cooking robot. Your possible action is grab, heat and plate",
    2: "I am a cleaning robot. Your possible action is clean and wipe",
    3: "I am a massage robot, Your possible action is go_to and give_massage"
}

class agumenter():
    def __init__(self):
        with open('./data/sample.json') as f:
            self.env = json.load(f)
    
    def reset(self):
        self.prompt = ""

    def get_prompt(self, task = 1, scene_num = 1):
        self.task = task
        self.prompt = PROMPT_STARTER[task]
        self.robot_name = "{} robot".format(AGENT_NAME[task])
        # self.prompt += "We have cooking robot, cleaning robot and massage robot.\n"
        # self.prompt += "I am a {}.\n".format(self.robot_name)
        self.prompt += "Here is the scene and the examples of the goal that {} can do.\n".format(self.robot_name)
        data = self.env[str(scene_num)]
        self.prompt += "objects : " + str(data['objects']) + "\n"
        if self.task != 1:
            self.prompt += "floorplan : " + str(data['floorplan']) + "\n"
            self.prompt += "people : " + str(data['people']) + "\n"
        agent_name = "agent_{}".format(self.task)
        goals = data[agent_name]['goal']
        for g in goals:
            self.prompt += "goal : " + g + "\n"
        self.prompt += "\n"
        
    def collect(self, label = 0):
        prompt = copy.deepcopy(self.prompt)
        if label == 0:
            prompt += "create the high level simple clear goal simlar to the examples.\n"
        elif label == 1:
            prompt += "create the creative goal that lacks the information to do the task but the robot still can do\n"
        elif label == 2:
            prompt += "create the simple goal that only {} cannot do\n".format(self.robot_name)
        elif label == 3:
            prompt += "create the goal that only {} cannot do with partially possible goal\n".format(self.robot_name)
            if self.robot_name == "cooking robot":
                prompt += 'for example, fix a broken table leg then serve water\n'
                prompt += 'for example, cook breakfast and solve the math problem\n'
            elif self.robot_name == "cleaning robot":
                prompt += 'for example, clean the bedroom then serve water\n'
                prompt += 'for example, cook dinner and wipe the table\n'
            elif self.robot_name == "massaging robot":
                prompt += 'for example, give massage wearing black shirt and then clean the room\n'
                prompt += 'for example, cook dinner and give massage to a person wearing blue shirt\n'
        prompt += "goal: "
        while True:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-4", 
                    messages=[{"role": "user", "content": prompt}], 
                    temperature = 0.8, top_p = 1, n = 3, stop="\n"
                    )
                break
            except:
                pass
        ans = []
        for r in response['choices']:
            temp = r['message']['content']
            if "cannot" in temp or "can't" in temp or "can not" in temp:
                continue
            ans.append(temp)
        print(ans)
        return ans

In [4]:
data = {}

In [5]:
aug = agumenter()
idx = 0
for scene in range(1, 13):
    for task in AGENT_NAME.keys():
        print(task)
        aug.reset()
        aug.get_prompt(task, scene)
        for label in range(3):
            for _ in range(20):
                resp = aug.collect(label)
                for ans in resp:
                    data[idx] = {"scene": {"floorplan":aug.env[str(scene)]['floorplan'],
                                            "objects": aug.env[str(scene)]['objects'],
                                            "people": aug.env[str(scene)]['people']}, 
                            "goal": ans, "label": label, "task": AGENT_NAME[task]}
                    idx += 1
    # break

1
['Prepare a meal or snack.', 'Prepare a meal or snack for the user based on their request.', 'Prepare breakfast.']
["Prepare a meal or beverage for the user's desired specification.", 'Prepare a meal or drink for the user.', 'Prepare a meal (using any of the objects available) and plate it.']
['Prepare a meal consisting of bacon and toast, with a side of water, and serve it on the table.', 'Prepare and plate a meal or snack.', 'Prepare a meal or beverage for the user.']
['Cook breakfast consisting of bacon and toast, and serve coffee.', 'Prepare a meal or drink for the user.', 'Prepare a meal or dish for the user.']
['Prepare a meal or beverage for the user.', 'Prepare a meal or dish for the user to eat.', 'cook bacon and toast bread for breakfast']
['Prepare breakfast.', 'Cook breakfast', 'Prepare breakfast.']


In [5]:
aug = agumenter()
idx = 5129
for scene in range(1, 13):
    for task in AGENT_NAME.keys():
        print(task)
        aug.reset()
        aug.get_prompt(task, scene)
        label = 2
        for _ in range(2):
            resp = aug.collect(3)
            for ans in resp:
                data[idx] = {"scene": {"floorplan":aug.env[str(scene)]['floorplan'],
                                        "objects": aug.env[str(scene)]['objects'],
                                        "people": aug.env[str(scene)]['people']}, 
                        "goal": ans, "label": label, "task": AGENT_NAME[task]}
                idx += 1
    # break

1
['Bake a cake and write a novel', 'wash the dishes then make a coffee', 'prepare a bacon sandwich and repair the television']
['bake bread and paint a picture', 'Prepare dinner and write a science research paper', 'brew coffee and repair a leaky faucet']
2
['Bake the bread and clean the kitchen', 'Bake bread and clean the kitchen', 'Prepare coffee and clean the living room']
['Prepare coffee and clean the living room', 'Bake bread then clean the kitchen', 'Cook bacon and clean the kitchen']
3
['Bake bread and give a massage to the person wearing a white shirt', 'Prepare coffee and give massage to the person wearing red shirt', 'Prepare coffee and give a massage to the person wearing white shirt.']
['Prepare coffee and give massage to a person wearing red shirt', 'Prepare coffee and give massage to person wearing red shirt', 'Prepare breakfast with bacon and bread and then give a massage to the person wearing a red shirt.']
1
['Bake a loaf of bread and write a novel', 'Clean the entir

In [6]:
with open('./data/agument.json', 'w') as f:
    json.dump(data, f, indent=4)

## Analyze

In [3]:
with open('./data/agument.json', 'r') as f:
    data = json.load(f)

In [8]:
res ={
    'cooking': [0,0,0],
    'cleaning': [0,0,0],
    'massaging': [0,0,0]
}
for val in data.values():
    task = val['task']
    label = val['label']
    if label == 3: continue
    # print(task, label)
    res[task][label] += 1
print(res)

{'cooking': [434, 636, 641], 'cleaning': [805, 298, 643], 'massaging': [510, 626, 629]}
